In [1]:
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [166]:
SCREEN_WIDTH = 1900
SCREEN_HEIGHT = 1200

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
COLORS = {
    "Red": (255, 0, 0),
    "Blue": (100, 100, 255),
    "Green": (0, 255, 0),
    "Yellow": (255, 255, 0),
}
CARD_WIDTH = 40
CARD_HEIGHT = 60

GRID_WIDTH = 500
GRID_HEIGHT = 80

HANDS_REGION = 200

# board region
BOARD_WIDTH = SCREEN_WIDTH - 2 * HANDS_REGION
BOARD_HEIGHT = SCREEN_HEIGHT - 2 * HANDS_REGION

In [223]:
class Card(pygame.sprite.Sprite):
    def __init__(self, color, number):
        super().__init__()

        self.color = color
        self.number = number
        self.rect = pygame.Rect(0, 0, CARD_WIDTH, CARD_HEIGHT)

        self.surface = pygame.Surface(self.rect.size)  # create a surface object
        self.render_to_surface()  # render card to surface


    def render_to_surface(self):
        card_color = COLORS.get(self.color, BLACK)
        cardBack = pygame.image.load("svg_playing_cards/backs/png_96_dpi/blue.png")
        self.surface.fill(card_color)  
        font = pygame.font.SysFont("arial", 28, bold=True, italic=False)
        text = font.render(f"{self.number}", True, WHITE, card_color)  # render number
        text_rect = text.get_rect(center=(self.rect.width / 2, self.rect.height / 2))
        self.surface.blit(text, text_rect)  # draw number on card
        
    def __str__(self) -> str:
        return f"({self.color}, {self.number})"
    
    def __repr__(self) -> str:
        return self.__str__()

In [224]:
def draw_grid(screen):
    for x in range(HANDS_REGION, HANDS_REGION + BOARD_WIDTH+1, GRID_WIDTH):
        pygame.draw.line(
            screen, BLACK, (x, HANDS_REGION), (x, HANDS_REGION + BOARD_HEIGHT)
        )
    for y in range(HANDS_REGION, HANDS_REGION + BOARD_HEIGHT+1, GRID_HEIGHT):
        pygame.draw.line(
            screen, BLACK, (HANDS_REGION, y), (HANDS_REGION + BOARD_WIDTH, y)
        )


def find_nearest_grid_pos(x, y):
    x_board = x - HANDS_REGION
    y_board = y - HANDS_REGION
    
    grid_x = x_board // GRID_WIDTH * GRID_WIDTH + CARD_WIDTH//2 + HANDS_REGION
    grid_y = y_board // GRID_HEIGHT * GRID_HEIGHT + GRID_HEIGHT // 2 + HANDS_REGION
    return grid_x, grid_y

def find_nearest_grid(x, y):
    x_board = x - HANDS_REGION
    y_board = y - HANDS_REGION
    
    col = x_board // GRID_WIDTH
    row = y_board // GRID_HEIGHT
    return row, col

In [225]:
def main():
    pygame.init()
    clock = pygame.time.Clock()

    pygame.display.set_caption("Rummikub Game")
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    all_cards = [Card("Red", 10), Card("Blue", 5), Card("Green", 8)]

    grid_cards = {}  # { (row, col): [card1, card2, ...], ... }

    dragging = False
    card_being_dragged = None
    offset_x = 0
    offset_y = 0

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # left button clicked
                    for card in all_cards:
                        if card.rect.collidepoint(event.pos):
                            dragging = True
                            card_being_dragged = card
                            
                            # record the original position of the card
                            original_row, original_col = find_nearest_grid(card.rect.centerx, card.rect.centery)
                            original_grid = (original_row, original_col)
                            
                            offset_x = card.rect.x - event.pos[0]
                            offset_y = card.rect.y - event.pos[1]
                            break

            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1 and dragging:  # left button released
                    
                    row, col = find_nearest_grid(card_being_dragged.rect.centerx, card_being_dragged.rect.centery)
                    
                    if original_grid in grid_cards and card_being_dragged in grid_cards[original_grid]:
                        grid_cards[original_grid].remove(card_being_dragged)
                    
                    if (row, col) not in grid_cards:
                        grid_cards[(row, col)] = []
                    
                    grid_x, grid_y = find_nearest_grid_pos(card_being_dragged.rect.centerx, card_being_dragged.rect.centery)
                    
                    # if the card is outside the board, put it back to the original position
                    if row < 0 or row >= BOARD_HEIGHT // GRID_HEIGHT or col < 0 or col >= BOARD_WIDTH // GRID_WIDTH:
                        row, col = original_row, original_col
                    else:
                        # update the card's position
                        card_being_dragged.rect.centerx = grid_x + CARD_WIDTH * len(grid_cards[(row, col)]) + 5*len(grid_cards[(row, col)])+5
                        card_being_dragged.rect.centery = grid_y
                    
                        grid_cards[(row, col)].append(card_being_dragged)
                    
                    print(grid_cards)
                    dragging = False
                    card_being_dragged = None

            elif event.type == pygame.MOUSEMOTION:
                if dragging and card_being_dragged is not None:
                    mouse_x, mouse_y = event.pos
                    new_x = mouse_x + offset_x
                    new_y = mouse_y + offset_y

                    # make sure the card is within the screen region
                    new_x = max(0, new_x)
                    new_y = max(0, new_y)
                    
                    new_x = min(SCREEN_WIDTH - card_being_dragged.rect.width, new_x)
                    new_y = min(SCREEN_HEIGHT - card_being_dragged.rect.height, new_y)

                    # update the card's position
                    card_being_dragged.rect.x = new_x
                    card_being_dragged.rect.y = new_y

        screen.fill('aquamarine4')
        draw_grid(screen)

        for card in all_cards:
            screen.blit(card.surface, card.rect)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

In [226]:
main()

{(0, 0): [(Red, 10)]}
{(0, 0): [(Red, 10), (Blue, 5)]}
{(0, 0): [(Red, 10), (Blue, 5)], (1, 0): [(Green, 8)]}
